In [42]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [43]:
# Load filtered training data
tr_msAcc = np.load("train_MSAccelerometer_OpenDoor_RubHands.npy")
tr_msGyr = np.load("train_MSGyroscope_OpenDoor_RubHands.npy")
tr_labels = np.load("train_labels_OpenDoor_RubHands.npy")

# Load filtered testing data
ts_msAcc = np.load("test_MSAccelerometer_OpenDoor_RubHands.npy")
ts_msGyr = np.load("test_MSGyroscope_OpenDoor_RubHands.npy")
ts_labels = np.load("test_labels_OpenDoor_RubHands.npy")

# Check the shapes of the loaded data
print("Training Accelerometer shape:", tr_msAcc.shape)
print("Training Gyroscope shape:", tr_msGyr.shape)
print("Training Labels shape:", tr_labels.shape)

print("Testing Accelerometer shape:", ts_msAcc.shape)
print("Testing Gyroscope shape:", ts_msGyr.shape)
print("Testing Labels shape:", ts_labels.shape)


Training Accelerometer shape: (87, 268, 3)
Training Gyroscope shape: (87, 268, 3)
Training Labels shape: (87,)
Testing Accelerometer shape: (90, 268, 3)
Testing Gyroscope shape: (90, 268, 3)
Testing Labels shape: (90,)


In [34]:
# Set number of training and testing examples
train_num_examples = 87
test_num_examples = 90

num_sensors = 2
num_features_per_axis = 8
num_axes = 3

train_features = np.zeros((train_num_examples, num_sensors * num_features_per_axis, num_axes))
test_features = np.zeros((test_num_examples, num_sensors * num_features_per_axis, num_axes))

In [44]:
# Generation of features for training dataset
# Accelerometer features
train_features[:, 0, :] = np.mean(tr_msAcc, axis=1)
train_features[:, 1, :] = np.max(tr_msAcc, axis=1)
train_features[:, 2, :] = np.min(tr_msAcc, axis=1)
train_features[:, 3, :] = np.std(tr_msAcc, axis=1)
train_features[:, 4, :] = np.sum(np.diff(np.sign(tr_msAcc), axis=1) != 0, axis=1)
train_features[:, 5, :] = np.percentile(tr_msAcc, 20, axis=1)
train_features[:, 6, :] = np.percentile(tr_msAcc, 50, axis=1)
train_features[:, 7, :] = np.percentile(tr_msAcc, 80, axis=1)

# Gyroscope features
train_features[:, 8, :] = np.mean(tr_msGyr, axis=1)
train_features[:, 9, :] = np.max(tr_msGyr, axis=1)
train_features[:, 10, :] = np.min(tr_msGyr, axis=1)
train_features[:, 11, :] = np.std(tr_msGyr, axis=1)
train_features[:, 12, :] = np.sum(np.diff(np.sign(tr_msGyr), axis=1) != 0, axis=1)
train_features[:, 13, :] = np.percentile(tr_msGyr, 20, axis=1)
train_features[:, 14, :] = np.percentile(tr_msGyr, 50, axis=1)
train_features[:, 15, :] = np.percentile(tr_msGyr, 80, axis=1)

# Generation of features for testing dataset
# Accelerometer features
test_features[:, 0, :] = np.mean(ts_msAcc, axis=1)
test_features[:, 1, :] = np.max(ts_msAcc, axis=1)
test_features[:, 2, :] = np.min(ts_msAcc, axis=1)
test_features[:, 3, :] = np.std(ts_msAcc, axis=1)
test_features[:, 4, :] = np.sum(np.diff(np.sign(ts_msAcc), axis=1) != 0, axis=1)
test_features[:, 5, :] = np.percentile(ts_msAcc, 20, axis=1)
test_features[:, 6, :] = np.percentile(ts_msAcc, 50, axis=1)
test_features[:, 7, :] = np.percentile(ts_msAcc, 80, axis=1)

# Gyroscope features
test_features[:, 8, :] = np.mean(ts_msGyr, axis=1)
test_features[:, 9, :] = np.max(ts_msGyr, axis=1)
test_features[:, 10, :] = np.min(ts_msGyr, axis=1)
test_features[:, 11, :] = np.std(ts_msGyr, axis=1)
test_features[:, 12, :] = np.sum(np.diff(np.sign(ts_msGyr), axis=1) != 0, axis=1)
test_features[:, 13, :] = np.percentile(ts_msGyr, 20, axis=1)
test_features[:, 14, :] = np.percentile(ts_msGyr, 50, axis=1)
test_features[:, 15, :] = np.percentile(ts_msGyr, 80, axis=1)

print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)


Train features shape: (87, 16, 3)
Test features shape: (90, 16, 3)


In [45]:
train_features_reshaped = np.reshape(train_features, (train_features.shape[0], train_features.shape[1] * train_features.shape[2]))
test_features_reshaped = np.reshape(test_features, (test_features.shape[0], test_features.shape[1] * test_features.shape[2]))

print("Train features reshaped shape:", train_features_reshaped.shape)
print("Test features reshaped shape:", test_features_reshaped.shape)

Train features reshaped shape: (87, 48)
Test features reshaped shape: (90, 48)


In [46]:
train_features_tensor = torch.from_numpy(train_features_reshaped).float()
test_features_tensor = torch.from_numpy(test_features_reshaped).float()
train_labels_tensor = torch.from_numpy(tr_labels).float().view(-1, 1)
test_labels_tensor = torch.from_numpy(ts_labels).float().view(-1, 1)

batch_size = 50
learning_rate = 0.01
epochs = 150

#OPEN_DOOR == 0
#RUB_HANDS == 1

#Create DataLoader for batch processing
train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_features_tensor, test_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

input_size = train_features_reshaped.shape[1]  # 48 features
model = BinaryClassificationModel(input_size)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

# Train the model
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluate the model on the test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # Convert sigmoid output to binary (0 or 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Epoch [1/150], Loss: 1.8625
Epoch [2/150], Loss: 1.4653
Epoch [3/150], Loss: 0.7339
Epoch [4/150], Loss: 0.6014
Epoch [5/150], Loss: 0.5278
Epoch [6/150], Loss: 0.3350
Epoch [7/150], Loss: 0.2677
Epoch [8/150], Loss: 0.6972
Epoch [9/150], Loss: 1.6616
Epoch [10/150], Loss: 0.5500
Epoch [11/150], Loss: 0.4508
Epoch [12/150], Loss: 0.3408
Epoch [13/150], Loss: 0.2718
Epoch [14/150], Loss: 0.2587
Epoch [15/150], Loss: 0.2061
Epoch [16/150], Loss: 0.2136
Epoch [17/150], Loss: 0.2287
Epoch [18/150], Loss: 0.1648
Epoch [19/150], Loss: 0.1777
Epoch [20/150], Loss: 0.1494
Epoch [21/150], Loss: 0.1070
Epoch [22/150], Loss: 0.0884
Epoch [23/150], Loss: 0.0734
Epoch [24/150], Loss: 0.0698
Epoch [25/150], Loss: 0.1271
Epoch [26/150], Loss: 0.1513
Epoch [27/150], Loss: 0.1100
Epoch [28/150], Loss: 0.1545
Epoch [29/150], Loss: 0.0695
Epoch [30/150], Loss: 0.0697
Epoch [31/150], Loss: 0.1050
Epoch [32/150], Loss: 0.0512
Epoch [33/150], Loss: 0.0720
Epoch [34/150], Loss: 0.0302
Epoch [35/150], Loss: 0

In [47]:
log_reg_classifier = LogisticRegression(solver='liblinear')
log_reg_classifier.fit(train_features_reshaped, tr_labels)

#Predict on the test data
estimatedLabels = log_reg_classifier.predict(test_features_reshaped)

#Evaluate the performance
accuracy = accuracy_score(ts_labels, estimatedLabels)
weightedF1 = f1_score(ts_labels, estimatedLabels, average='weighted')
averageF1 = f1_score(ts_labels, estimatedLabels, average='macro')
allF1Scores = f1_score(ts_labels, estimatedLabels, average=None)
conf_matrix = confusion_matrix(ts_labels, estimatedLabels)

print('Logistic Regression Results:')
print('   Average F1-score = %.4f' % (averageF1))
print('   Test accuracy = %.2f %%' % (accuracy * 100))
print('   Weighted F1-score = %.4f' % (weightedF1))
print('   All F1-scores:', allF1Scores)
print('   Confusion Matrix:')
print(conf_matrix)
print('-------------------------------------------------------')

Logistic Regression Results:
   Average F1-score = 0.9333
   Test accuracy = 93.33 %
   Weighted F1-score = 0.9334
   All F1-scores: [0.93478261 0.93181818]
   Confusion Matrix:
[[43  5]
 [ 1 41]]
-------------------------------------------------------
